In [ ]:
%pylab inline
import h5py
# import fitsio
import astropy.units as u
from astropy.io import fits
from astropy.table import Table
from astropy.visualization import hist
from astropy import coordinates as coords
from scipy import stats
import networkx as nx

import gwb

In [ ]:
style.use(['seaborn-colorblind', 'notebook.mplstyle'])

# Load Data

In [ ]:
tgas = gwb.TGASData('../data/stacked_tgas.fits')

In [ ]:
pairidx_rand = fits.getdata('../output/random/snr8_random200000.fits')
with h5py.File("../output/random/snr8_random200000_vscatter0-lratio.h5") as f:
    lnH1_rand = f['lnH1'].value
    lnH2_rand = f['lnH2'].value
    llr_rand = lnH1_rand - lnH2_rand
# throw out nans
bad = isnan(llr_rand)
pairidx_rand = pairidx_rand[~bad]
lnH1_rand = lnH1_rand[~bad]
lnH2_rand = lnH2_rand[~bad]
llr_rand = llr_rand[~bad]

pairidx = fits.getdata('../output/23560/snr8_r10_dv10.fits')
with h5py.File("../output/23560/snr8_r10_dv10_vscatter0-lratio.h5") as f:
    lnH1 = f['lnH1'].value
    lnH2 = f['lnH2'].value
    llr = lnH1 - lnH2

In [ ]:
# put quantities in arrays
parallax_snr = tgas.parallax_snr
vtan = tgas.get_vtan().value
c = tgas.get_coord()
d = tgas.get_distance().value

star1, star2 = pairidx['star1'], pairidx['star2']
min_snr = np.min(np.vstack((parallax_snr[star1], parallax_snr[star2])), axis=0)
dvtan = norm(vtan[star1]-vtan[star2], axis=1)
vtanmean = (vtan[star1] + vtan[star2])*0.5
sep = c[star1].separation_3d(c[star2]).value
sep_sky = c[star1].separation(c[star2])

c1 = c[star1]
c2 = c[star2]
ra1, dec1 = c1.ra.value, c1.dec.value
ra2, dec2 = c2.ra.value, c2.dec.value
l1, b1 = c1.transform_to(coords.Galactic).l.value, c1.transform_to(coords.Galactic).b.value
l2, b2 = c2.transform_to(coords.Galactic).l.value, c2.transform_to(coords.Galactic).b.value
d1 = d[star1]
d2 = d[star2]
dmean = (d1+d2)*0.5

In [ ]:
print('total number of pairs examined =', pairidx.size)

# Figure: Likelihood ratio histogram

In [ ]:
plot(pairidx['sep'], pairidx['delta_v'], 'k.', alpha=0.2, ms=3, rasterized=True)
xlabel(r'separation [pc]')
yl = ylabel(r'$|\Delta \boldsymbol{v}_{\rm{t}}|$ [km s$^{-1}$]')

from astropy.constants import G
tmpsep = logspace(-3,1)
v = sqrt(G*(2*u.solMass)/(tmpsep*u.pc)).to(u.km/u.s).value
plot(tmpsep, v, label=r"$v_\mathrm{orb} (a)$"
     "\n"
     r"(assuming $M=2\,M_\odot$)", lw=2)
legend(loc='upper left', fontsize=15, frameon=False)

xscale('log')
_ = yticks([0,2,4,6,8,10])
tight_layout()
savefig('../paper/figures/sep_dvtan.pdf', dpi=150)

# Figure: likelihood ratio / sep-dvtan (NEW)

In [ ]:
v_cuts = [(0, 2.5), (2.5, 5), (5, 7.5), (7.5, 10)]
v_colors = ['#7a0177', '#c51b8a', '#f768a1', '#fbb4b9']

llr_cuts = [(-6, -2), (-2, 2), (2, 6), (6, 10)]
llr_colors = ['#bae4bc', '#7bccc4', '#43a2ca', '#0868ac']

In [ ]:
fig ,ax = plt.subplots(2,2,figsize=(8,6), sharex='col')
fig.subplots_adjust(left=0.1,bottom=0.12,top=0.97,right=0.97, hspace=0.07, wspace=0.24)
ax[1,0].hist(llr,
             bins=np.linspace(-6, 10, 64),
             histtype='step', normed=True, linewidth=2,
             color='k')

_ = ax[1,0].hist(llr_rand, bins=linspace(-6,10,64),
         color='0.8', normed=True, zorder=-1,
         label='random pairs')

ax[1,0].set_xlim(-6,10)
ax[1,0].set_ylim(0, 0.36)
ax[1,0].set_xlabel(r"$\ln(L_1/L_2)$")
ax[1,0].set_ylabel("density")

ax[1,1].set_xlim(4E-3, 1E1)
ax[1,1].set_ylim(0, 10)
ax[1,1].set_xlabel(r'separation [pc]')
ax[1,1].set_xscale('log')
ax[1,1].set_ylabel(r'$|\Delta \boldsymbol{v}_{\rm{t}}|$ [km s$^{-1}$]')

for i,(l,r) in enumerate(llr_cuts):
    ax[1,0].fill_betweenx([ax[1,0].get_ylim()[1]*0.9,1.], l, r,
                          color=llr_colors[i], zorder=-100)
    idx = ((llr) > l) & (llr < r)
    ax[1,1].plot(pairidx['sep'][idx], pairidx['delta_v'][idx],
                 color=llr_colors[i], 
                 linestyle='none', marker='.', alpha=0.3, ms=3,
                 markeredgecolor='none', markeredgewidth=0, rasterized=True)

_ = ax[0,0].hist(llr_rand, bins=linspace(-6,10,64),
         color='0.8', normed=True, zorder=-1,
         label='random pairs')
ax[0,1].plot(pairidx['sep'], pairidx['delta_v'],
             color='k', linestyle='none', marker='.', zorder=-10, alpha=0.3, ms=3, rasterized=True)

ax[0,0].set_xlim(-6,10)
ax[0,0].set_ylim(0, 0.36)
ax[0,0].set_ylabel("density")
ax[0,0].legend(loc='upper left', frameon=False, fontsize=15)

ax[0,1].set_xlim(4E-3, 1E1)
ax[0,1].set_ylim(0, 10)
ax[0,1].set_xscale('log')
ax[0,1].set_ylabel(r'$|\Delta \boldsymbol{v}_{\rm{t}}|$ [km s$^{-1}$]')

for i,(l,r) in enumerate(v_cuts):
    xlims = ax[0,1].get_xlim()
    xp10 = xlims[0]*2
    ax[0,1].fill_between([0,xp10], l, r, color=v_colors[i], zorder=-100)

    idx = (pairidx['delta_v'] > l) & (pairidx['delta_v'] < r)
    ax[0,0].hist(llr[idx], bins=np.linspace(-6, 10, 64), 
                 histtype='step', normed=True, color=v_colors[i], linewidth=2);
        
# savefig('../paper/figures/likelihoodratios.pdf')

In [ ]:
cond_lr_cut = llr>6
print('total number of pairs selected =', cond_lr_cut.sum(), 'sep < 1pc =', sum((sep<1)&cond_lr_cut))
cmpairs = pairidx[cond_lr_cut]

# Examine the network of pairs

In [ ]:
graph = nx.Graph()
graph.add_edges_from(
    [(i,j, {'lnL1/L2':ll}) for (i,j,dvtan,s), ll in zip(cmpairs, llr[llr>6])])

In [ ]:
connected = array([array(list(c)) for c in nx.connected_components(graph)])
sizes = array([len(c) for c in nx.connected_components(graph)])
print('number of nodes %i' % (len(graph)))
print('total number of connected components %i' % (connected.size))
print(min(sizes),max(sizes))

In [ ]:
nn_nodes = array([len(graph.neighbors(i)) for i in graph.nodes()])
print('most connected star ind %i connection size %i' % (graph.nodes()[nn_nodes.argmax()], nn_nodes.max()))
# print(tgas[graph.nodes()[nn_nodes.argmax()]]._data)

In [ ]:
tmass = fits.getdata('../data/tgas_tmassj.fits', ext=1)
jhk = Table.read('result.vot', format='votable')

In [ ]:
mwsc = Table.read('../data/J_A+A_585_A101/catalog.dat', readme='../data/J_A+A_585_A101/ReadMe',
                 format='ascii.cds')
print('total number of mwsc', len(mwsc))
print('number of mwsc d<600 pc', (mwsc['d']<600).sum())

In [ ]:
components = sorted(nx.connected_component_subgraphs(graph), key=len, reverse=True)

comp_dict = {idx: comp.nodes() for idx, comp in enumerate(components)}
attr = {n: comp_id for comp_id, nodes in comp_dict.items() for n in nodes}

nx.set_node_attributes(graph, "group_id", attr)
print(graph.nodes(data=True)[:3])

## table stub

In [ ]:
stub = tt['star1 source id', 'star2 source id', 'sep', 'lnL1/L2', 'nid', 'nsize'][:10]

In [ ]:
stub['sep'].name = 'separation'
stub['separation'].unit = u.pc
stub['lnL1/L2'].name = '$\ln \mathcal{L}_1 /\mathcal{L}_2$'
stub['nid'].name = 'ID$_\mathrm{network}$'
stub['nsize'].name = '$N_\mathrm{network}$'

In [ ]:
caption = r'Catalog of candidate co-moving pairs \label{tab:catalog}'
tablefoot = r'\tablecomments{Table 1 is published in its entirety in the machine-readable format. A portion is shown here for guidance regarding its form and content.}'

stub.write('../paper/stub.tex', format='ascii.aastex',
           caption=caption, latexdict=dict(tablefoot=tablefoot))

In [ ]:
counts,_,_ = hist(sizes, bins=arange(1.5,10.6,1), log=True,
                  histtype='bar', color='k', facecolor='None', edgecolor='k', lw=1.5)
print(counts.sum())
xlim(1.5, 11.5)
xticks([2,3,4,5,6,7,8,9,10,11])
xlabel('size of connected component')
ylabel('count')
text(0.95, 0.95, '%i with size $>$ 10 not shown' % ((sizes>10).sum()),
     transform=gca().transAxes, va='top', ha='right', size=15)

# [i.set_linewidth(2) for i in gca().spines.values()]
# gca().tick_params(axis='both', which='major', size=8)
# gca().tick_params(axis='both', which='minor', size=4)
tight_layout()
savefig('../paper/figures/dist_networksize.pdf')

In [ ]:
# sorted list of subgraphs from largest to smallest
Gc = array(sorted(nx.connected_component_subgraphs(graph), key=len, reverse=True))
sizes = array([len(g) for g in Gc])

In [ ]:
# graph visualization of selected groups
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.gridspec as gridspec

def set_line_alpha(ax):
    linecollection = ax.collections[1]
    linecollection.set_alpha(0.5)

fig = figure(figsize=(10,4))
fig.subplots_adjust(top=0.95, right=0.98, bottom=0.17, left=0.08)
ax0 = subplot(121)
g = Gc[0]
# pos = {node:(tgas.l[node], d[node]) for node in g.nodes()}
pos = {node:(tgas.ra.value[node], tgas.dec.value[node]) for node in g.nodes()}
nx.draw_networkx(g, pos=pos, node_size=10, width=.5, with_labels=False, alpha=1)
set_line_alpha(ax0)
xlabel(r'$\alpha$ [deg]')
ylabel(r'$\delta$ [deg]')
text(0.05, 0.05, 'size=%i\n%i edges' % (len(g), len(g.edges())),
     transform=gca().transAxes, va='bottom', ha='left', size=20)

grid = gridspec.GridSpec(2,2)
grid.update(left=0.54, right=0.98, wspace=0.25)
ax = [subplot(i) for i in grid]
for i, gi in enumerate([6,13,37,51]):
    g = Gc[gi]
#     pos = {node:(tgas.l[node], d[node]) for node in g.nodes()}
    pos = {node:(tgas.ra.value[node], tgas.dec.value[node]) for node in g.nodes()}
    nx.draw_networkx(g, pos=pos, node_size=10, width=.5, with_labels=False, ax=ax[i])
    ax[i].xaxis.set_major_locator(MaxNLocator(nbins=5,prune='both'))
    ax[i].yaxis.set_major_locator(MaxNLocator(nbins=5,prune='both'))
    ax[i].text(0.95, 0.05, 'size=%i\n%i edges' % (len(g), len(g.edges())),
               transform=ax[i].transAxes, va='bottom', ha='right', size=15)
    ax[i].tick_params(which='both', labelsize=15)
    set_line_alpha(ax[i])

ax[0].set_xticks([128, 130, 132])
# savefig('../paper/figures/graphviz_examples.pdf')
# savefig('../paper/figures/graphviz_examples.png', dpi=150)

In [ ]:
mwsc = Table.read('../data/J_A+A_585_A101/catalog.dat', readme='../data/J_A+A_585_A101/ReadMe',
                 format='ascii.cds')
print('total number of mwsc', len(mwsc))
print('number of mwsc d<600 pc', (mwsc['d']<600).sum())

In [ ]:
obass = Table.read('../data/J_AJ_117_354/tablec1.dat', readme='../data/J_AJ_117_354/ReadMe',
                    format='ascii.cds')
print('OB association stars', len(obass))

from astroquery.simbad import Simbad

# query simbad on HIP id's to get coordinates
customSimbad = Simbad()
customSimbad.add_votable_fields('sptype', 'parallax')
result = customSimbad.query_objects(['HIP %i' % hip for hip in obass['HIP']])
print( unique([s.decode("utf-8")[0] if len(s)>0 else '?' for s in result['SP_TYPE']]) )

def get_distance(parallax, parallax_error):
    """
    Return the distance [kpc] point estimate with the Lutz-Kelker correction
    
    parallax : float, in mas
    parallax_error : float, in mas
    """
    snr = parallax / parallax_error
    pnew = parallax * (0.5 + 0.5*np.sqrt(1 - 16./snr**2))
    # if snr<4, the value will be maksed
    return 1./pnew

obass_dist = get_distance(result['PLX_VALUE'], result['PLX_ERROR'])
obass_c = coords.SkyCoord(result['RA'], result['DEC'], unit=(u.hourangle, u.deg),
                          distance=obass_dist*u.kpc)
obass_cg = obass_c.transform_to(coords.Galactic)

In [ ]:
import palettable
from itertools import cycle
colorloop = cycle(palettable.tableau.Tableau_20.hex_colors)

fig = figure(figsize=(10,10))
fig.subplots_adjust(top=0.95,bottom=0.04,right=0.93,left=0.05)

ax = subplot(111)
ax.set_color_cycle(palettable.tableau.Tableau_20.mpl_colors)

mwsc_close = mwsc[mwsc['d']<600]
dx = 10
clustercircles = [Circle((tmp.x.value, tmp.y.value), dx, zorder=-50,
                         facecolor='#428bca', alpha=.5)
                  for tmp in mwsc_cg.cartesian]
[ax.add_patch(cc) for cc in clustercircles]

h2, = ax.plot(obass_cg.cartesian.x.to(u.pc).value,
              obass_cg.cartesian.y.to(u.pc).value,
              c='b', mec='#7059ff', mew=1., marker='+', ls='None', zorder=-100)
h1 = Line2D((0,1), (0,0), marker='o', linestyle='', ms=10, mfc='#428bca', alpha=.5)

for g in Gc[::]:
    if len(g) > 5:
        pos = {node:(gx[node], gy[node])
               for node in g.nodes()}
        nx.draw_networkx(g, pos=pos, node_size=3, width=2, with_labels=False,
#                          edge_color=mpl.colors.rgb2hex(cm.spectral(random.uniform())),
                         edge_color=next(colorloop),
                         node_color='k')
    else:
        pos = {node:(gx[node], gy[node])
               for node in g.nodes()}
        nx.draw_networkx_edges(g, pos=pos, width=1, with_labels=False,
                               edge_color='k',alpha=.5)
        



legend = figlegend((h1, h2), ('star clusters ($r=10$ pc)', 'stars in OB Associations'), 'upper right',
    numpoints=1, frameon=True, fontsize=20,
    handletextpad=0, markerscale=2, scatterpoints=1, )
legend.get_frame().set_facecolor('white')

ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.set_xticks([])
ax.set_yticks([])

gridcircles = [Circle((0, 0), r, facecolor='None', linestyle='dotted', ec='k')
               for r in [100, 200, 300]]
[ax.add_patch(cc) for cc in gridcircles]
[ax.text(r*cos(pi/4), r*sin(pi/4), '%.0f pc' % (r), size=20) for r in [100,200,300]]

ax.axhline(0., c='k', ls='dotted')
ax.axvline(0., c='k', ls='dotted')

ax.set_xlim(-300,300)
ax.set_ylim(-300,350)

savefig('../paper/figures/glon_d_pie.pdf', dpi=150, )

# close()

In [ ]:
rave = Table.read('../data/tgas_rave_RV.fits')

In [ ]:
matched_rave = ~rave['row_id'].mask
print('total number of tgas+rave matches', matched_rave.sum())

hrv = rave['HRV']
ehrv = rave['eHRV'].data

# rv1 = hrv[pairidx['star1']]
# rv2 = hrv[pairidx['star2']]
# drv = rv1 - rv2
# edrv = hypot(ehrv[pairidx['star1']], ehrv[pairidx['star2']])
# matched_rave1 = matched_rave[pairidx['star1']]
# matched_rave2 = matched_rave[pairidx['star2']]
# matched_rave12 = matched_rave1 & matched_rave2

In [ ]:
star1 = cmpairs['star1']
star2 = cmpairs['star2']
nsize = array([len(nx.node_connected_component(graph, s1)) for s1 in star1])

In [ ]:
((nsize==2) & (cmpairs['sep']>3)).sum()

In [ ]:
drv = hrv[star1] - hrv[star2]
edrv = hypot(ehrv[star1], ehrv[star2])
drv_edrv = drv/edrv
cmrave = matched_rave[star1] & matched_rave[star2]

In [ ]:
print('number of pairs with RAVE', prod(matched_rave[vstack([star1,star2]).T], axis=1).sum())
print('number of mutually exclusive pairs with RAVE',
      prod(matched_rave[vstack([star1,star2]).T[nsize==2]], axis=1).sum())

In [ ]:
import pandas as pd
from scipy import interpolate

In [ ]:
# separation vs RV difference
# colors for two groups
color_size2 = '#d95f02'
color_sizelt2 = '#1b9e77'

fig, ax = subplots(2, 1, figsize=(5, 8), sharex=True)
fig.subplots_adjust(top=0.93, hspace=0.05, left=0.2, bottom=0.08, right=0.95)
suptitle('%i pairs with RAVE RV' % cmrave.sum(), size=20, x=0.55)

sca(ax[0])
errorbar(cmpairs['sep'][cmrave & (nsize==2)], drv_edrv[cmrave & (nsize==2)],
         fmt='.',color=color_size2, label='size=2 (mutually exclusive)')
errorbar(cmpairs['sep'][cmrave & (nsize>2)], drv_edrv[cmrave & (nsize>2)],
         fmt='^', mew=1, ms=4, mec=color_sizelt2, mfc='None', label='size$>$2',)
axhline(-3, color='gray')
axhline(3, color='gray')
ylim(-20,20)
# xscale('log')
xlim(0,10)
ylabel('$\Delta v_r/\sigma_{\Delta v_r}$')

legend(loc=(-0.03,0.75), frameon=False, numpoints=1, markerscale=2, fontsize=15, handletextpad=0)

sca(ax[1])
n=15  # each bin contains 2n+1 data points

tmpsep = pd.Series(cmpairs['sep'][cmrave & (nsize==2)].astype('<f8'))
bool3 = pd.Series((drv_edrv<3)[cmrave & (nsize==2)].astype('<f8'))
isort = tmpsep.argsort()
tmpsep_sort = tmpsep[isort]
bool3_sort = bool3[isort]
rollsep = tmpsep_sort.rolling(2*n+1, center=True)
rollbool = bool3_sort.rolling(2*n+1, center=True)
errors = vstack([
    rollsep.median() - rollsep.min(),
    rollsep.max() - rollsep.median()]).T[n:-n]

errorbar(rollsep.median()[n:-n], rollbool.mean()[n:-n],
         xerr=errors.T, fmt='.', color=color_size2)

# fit spline
# x = log10(rollsep.median()[n:-n])
# y = rollbool.mean()[n:-n]
# tck = interpolate.splrep(x, y, s=1, k=3)
# xnew = logspace(-1, 1)
# ynew = interpolate.splev(log10(xnew), tck, der=0, ext=3)
# plot(xnew, ynew, 'b.-')

tmpsep = pd.Series(cmpairs['sep'][cmrave & (nsize>2)].astype('<f8'))
bool3 = pd.Series((drv_edrv<3)[cmrave & (nsize>2)].astype('<f8'))
isort = tmpsep.argsort()
tmpsep_sort = tmpsep[isort]
bool3_sort = bool3[isort]
rollsep = tmpsep_sort.rolling(2*n+1, center=True)
rollbool = bool3_sort.rolling(2*n+1, center=True)
errors = vstack([
    rollsep.median() - rollsep.min(),
    rollsep.max() - rollsep.median()]).T[n:-n]
errorbar(rollsep.median()[n:-n], rollbool.mean()[n:-n],
         xerr=errors.T, fmt='^', mew=1, ms=4, mec=color_sizelt2, mfc='None', ecolor=color_sizelt2, capsize=0)

# xscale('log')
ylim(0.,1.05)
xlim(0,10)
xlabel('separation [pc]')
ylabel('$f(|\Delta v_r/\sigma_{\Delta v_r}| < 3)$')
# savefig('../paper/figures/raverv.pdf')

In [ ]:
# separation distribution
from astropy import stats as astats
binsize = astats.knuth_bin_width(log10(cmpairs['sep']))
bins = logspace(-2,1,3/binsize)

c,bins,patches = hist(cmpairs['sep'], bins, log=True, histtype='step',
                      label='all', color='k', lw=1.5)
_ = hist(cmpairs['sep'][nsize==2], bins, log=True, histtype='step',
         label='size=2 (mutually exclusive)', lw=2.5, color=color_size2)
_ = hist(cmpairs['sep'][nsize>2], bins, log=True, histtype='step', label='size$>$2', color=color_sizelt2,
         lw=1.5)
ylabel('count')
xlabel('separation [pc]')
legend(loc='upper left', fontsize=15, frameon=False)
tight_layout()
xscale('log')
savefig('../paper/figures/hist_sep.pdf')

In [ ]:

counts, edges = histogram(log10(cmpairs['sep'][nsize==2]), 32, )
centers = (edges[1:]+edges[:-1])*0.5
ynew = interpolate.splev(centers, tck, der=0, ext=3)
plot(centers, counts, 'ko-')
plot(centers, counts*ynew, 'bo-')
yscale('log')

# Color-Magnitude Diagrams

In [ ]:
tmass = fits.getdata('../data/tgas_tmassj.fits', ext=1)

In [ ]:
print('number of tgas stars without 2mass J =', isnan(tmass['j_m']).sum())

In [ ]:
gjcolor = tgas._data['phot_g_mean_mag']- tmass['j_m']
gMag = tgas._data['phot_g_mean_mag'] + 5*(log10(tgas._data['parallax']*1e-3)+1)

In [ ]:
# distance matching sample
ind_uniq = unique(hstack([cmpairs['star1'], cmpairs['star2']]))
print(ind_uniq.size)
dpairs = d[ind_uniq]

In [ ]:
hist(dpairs)
xlabel('distance [pc]')
ylabel('count')

In [ ]:
from sklearn.neighbors import KDTree

In [ ]:
cond = (parallax_snr>8) & ~isnan(tmass['j_m'])
d_snr = d[cond]
ind_snr = arange(len(tgas))[cond]
dtree = KDTree(atleast_2d(d_snr).T)

In [ ]:
plot(dpairs, dtree.query_radius(atleast_2d(dpairs).T, 1.5, count_only=True), '.')
yscale('log')
xlabel('distance')
ylabel('N(stars within 1.5pc)')

In [ ]:
N_per_star = 10
result = dtree.query_radius(atleast_2d(dpairs).T, 1.5,)

In [ ]:
ind_control = []
for row in result:
    if len(row)==0:
        continue
    ind_control.append(ind_snr[choice(row, size=N_per_star)])
ind_control = hstack(ind_control)

In [ ]:
d[ind_control]

In [ ]:
c,bins,patches=hist(dpairs, normed=True, histtype='step')
xlabel('distance [pc]')
ylabel('count')
hist(d[ind_control], bins=bins, normed=True, histtype='step')

In [ ]:
plot(gjcolor[ind_control], gMag[ind_control], 'k,')

In [ ]:
c, ye, xe = histogram2d(gjcolor[ind_control], gMag[ind_control], 512,)
x,y = meshgrid((xe[1:]+xe[:-1])*0.5, (ye[1:]+ye[:-1])*0.5)
pcolormesh(y,x,c, norm=mpl.colors.LogNorm(), cmap='Blues', zorder=-1)


In [ ]:
def draw_cmd():
    c, ye, xe = histogram2d(gjcolor[ind_control], gMag[ind_control], 512,)
    x,y = meshgrid((xe[1:]+xe[:-1])*0.5, (ye[1:]+ye[:-1])*0.5)
    pcolormesh(y,x,c, norm=mpl.colors.LogNorm(), cmap='Blues', zorder=-1, rasterized=True)


In [ ]:
# plot color-magnitude diagram with control sample on the background
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.gridspec as gridspec

fig = figure(figsize=(10,4))
fig.subplots_adjust(top=0.95, right=0.98, bottom=0.18, left=0.08)
ax0 = subplot(121)
g = Gc[0]
pos = {node:(gjcolor[node],gMag[node]) for node in g.nodes()}
nx.draw_networkx(g, pos=pos, node_size=10, width=0, with_labels=False, node_color='k')
xlabel(r'$G-J$ [mag]')
ylabel(r'$G + 5(\log \hat d +1)$ [mag]')
text(0.05, 0.05, 'size=%i' % (len(g)),
     transform=gca().transAxes, va='bottom', ha='left', size=20)
draw_cmd()
ax0.set_xlim(-0.5, 2)
ax0.set_ylim(8,-1)

grid = gridspec.GridSpec(2,2)
grid.update(left=0.54, right=0.98)
ax = [subplot(i,) for i in grid]
for i, gi in enumerate([6,13,37,51]):
    g = Gc[gi]
    sca(ax[i])
    pos = {node:(gjcolor[node],gMag[node]) for node in g.nodes()}
    nx.draw_networkx(g, pos=pos, node_size=10, width=0, with_labels=False, ax=ax[i], node_color='k')
    ax[i].xaxis.set_major_locator(MaxNLocator(nbins=5,prune='both'))
    ax[i].yaxis.set_major_locator(MaxNLocator(nbins=5,prune='both'))
    ax[i].text(0.05, 0.05, 'size=%i' % (len(g)), size=15,
               transform=ax[i].transAxes, va='bottom', ha='left')
    ax[i].tick_params(which='both', labelsize=15)
    draw_cmd()

[a.set_xlim(-0.5,2.) for a in ax]
[a.set_ylim(8,-1) for a in ax]
# savefig('../paper/figures/gjg_graphviz_examples.pdf')
# savefig('../paper/figures/gjg_graphviz_examples.png', dpi=150)

In [ ]:
# plot mutually exclusive pairs in the color-magnitude diagram
fig, ax = subplots(1,2, figsize=(10,8))
fig.subplots_adjust(left=0.1, right=0.95, bottom=0.1, wspace=0.1)
suptitle('mutually exclusive pairs', size=20)

nsize = array([len(nx.node_connected_component(graph, s1)) for s1 in cmpairs['star1']])
mepairs = cmpairs[(nsize==2)]
print(mepairs.size)

star1,star2 = mepairs['star1'], mepairs['star2']
gj1, gj2 = gjcolor[star1], gjcolor[star2]
g1, g2 = gMag[star1], gMag[star2]

x = vstack([gj1,gj2,[None]*gj1.size]).T
y = vstack([g1,g2,[None]*g1.size]).T

sepdivide = 1  # pc

sca(ax[0])
draw_cmd()
l, = plot(x[mepairs['sep']<sepdivide].ravel(),y[mepairs['sep']<sepdivide].ravel(),
     'ko-', lw=1., mec='k', mew=0.5, ms=3, mfc='#0072B2')#, rasterized=True)
l.set_markerfacecolor((0, 114/256., 178/256., 0.5))

xlim(-.25,2.5)
ylim(10,-2)

xlabel(r'$G-J$ [mag]')
ylabel(r'$G + 5(\log \hat d +1)$ [mag]')
title('separation $<$ %.0f pc (N=%i)' % (sepdivide,(mepairs['sep']<sepdivide).sum()),
      size=20)

sca(ax[1])
draw_cmd()
plot(x[mepairs['sep']>sepdivide].ravel(),y[mepairs['sep']>sepdivide].ravel(),
     'k.', alpha=.5,)# rasterized=True)
xlim(-.25,2.5)
ylim(10,-2)
xlabel(r'$G-J$ [mag]')
title('separation $>$ %.0f pc (N=%i)' % (sepdivide,(mepairs['sep']>sepdivide).sum()),
      size=20)
savefig('../paper/figures/gjg_mepairs.pdf', dpi=150)

# Cross matching to known clusters / OB associations

In [ ]:
mwsc = Table.read('../data/J_A+A_585_A101/catalog.dat', readme='../data/J_A+A_585_A101/ReadMe',
                 format='ascii.cds')
print('total number of mwsc', len(mwsc))
print('number of mwsc d<600 pc', (mwsc['d']<600).sum())

In [ ]:
obass = Table.read('../data/J_AJ_117_354/tablec1.dat', readme='../data/J_AJ_117_354/ReadMe',
                    format='ascii.cds')
print('OB association stars', len(obass))

In [ ]:
from astroquery.simbad import Simbad

In [ ]:
# query simbad on HIP id's to get coordinates
customSimbad = Simbad()
customSimbad.add_votable_fields('sptype', 'parallax')
result = customSimbad.query_objects(['HIP %i' % hip for hip in obass['HIP']])
print( unique([s.decode("utf-8")[0] if len(s)>0 else '?' for s in result['SP_TYPE']]) )

In [ ]:
def get_distance(parallax, parallax_error):
    """
    Return the distance [kpc] point estimate with the Lutz-Kelker correction
    
    parallax : float, in mas
    parallax_error : float, in mas
    """
    snr = parallax / parallax_error
    pnew = parallax * (0.5 + 0.5*np.sqrt(1 - 16./snr**2))
    # if snr<4, the value will be maksed
    return 1./pnew

obass_dist = get_distance(result['PLX_VALUE'], result['PLX_ERROR'])

In [ ]:
obass_c = coords.SkyCoord(result['RA'], result['DEC'], unit=(u.hourangle, u.deg),
                          distance=obass_dist*u.pc)

In [ ]:
obass_cg = obass_c.transform_to(coords.Galactic)

In [ ]:
figure(figsize=(10,6))
# plot(d1[condllr], sep[condllr], '.', color='red', alpha=.5)
plot(d1[llr>6], sep[llr>6], '.', color='red', alpha=.2)
yscale('log')
xlabel('$d_1$ [pc]')
ylabel('separation [pc]')
ylim(0.1, 1.2e2)
xlim(0,600)
grid()

for cc in mwsc['d'][mwsc['d']<600]:
    axvline(cc, c='k', lw=1, ymin=0.2, ymax=0.4)
# savefig('d1_sep_mwsc.png')